<font color = red>Introduction to Business Analytics:<br>Using Python for Better Business Decisions</font>
=======
<br>
    <center><img src="http://dataanalyticscorp.com/wp-content/uploads/2018/03/logo.png"></center>
<br>
Taught by: 

* Walter R. Paczkowski, Ph.D. 

    * My Affliations: [Data Analytics Corp.](http://www.dataanalyticscorp.com/) and [Rutgers University](https://economics.rutgers.edu/people/teaching-personnel)
    * [Email Me With Questions](mailto:walt@dataanalyticscorp.com)
    * [Learn About Me](http://www.dataanalyticscorp.com/)
    * [See My LinkedIn Profile](https://www.linkedin.com/in/walter-paczkowski-a17a1511/)
    

# <font color = blue> Lesson \#3:<br>Predictive Modeling: Introduction to Machine Learning </font>

In this lesson, you will learn:

1. to divide your data set into training and testing sets;
2. estimate *OLS* and Logistic models; and
3. grow decision trees.

## <font color = black> Reset the Data from Lesson 1 </font>

Resetting the data will ensure that the work you did in Lesson 1 is available in this lesson.

In [ ]:
##
## Load packages
##
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
##
## Import the data.  The parse_dates argument says to 
## treat Tdate as a date object.
##
file = r'../Data/furniture/final data files/orders.csv'
df_orders = pd.read_csv( file, parse_dates = [ 'Tdate' ] )
pd.set_option('display.max_columns', 8)
##
## Initial Calculations
##
x = [ 'Ddisc', 'Odisc', 'Cdisc', 'Pdisc' ]
df_orders[ 'Tdisc' ] = df_orders[ x ].sum( axis = 1 )
##
df_orders[ 'Pprice' ] = df_orders.Lprice*( 1 - df_orders.Tdisc )
##
df_orders[ 'Rev' ] = df_orders.Usales * df_orders.Pprice
##
df_orders[ 'Con' ] = df_orders.Rev - df_orders.Mcost
df_orders[ 'CM' ] = df_orders.Con/df_orders.Rev
##
df_orders[ 'netRev' ] = ( df_orders.Usales - df_orders.returnAmount )*df_orders.Pprice
df_orders[ 'lostRev' ] = df_orders.Rev - df_orders.netRev
##
##
## Import a second DataFrame on the customers
##
file = r'../Data/furniture/final data files/customers.csv'
df_cust = pd.read_csv( file )
##
## Do an inner join using CID as the link
##
df = pd.merge( df_orders, df_cust, on = 'CID' )

## <font color = black> Steps for Predictive Modeling </font>

There are three steps for predictive modeling:

1. Split your data into two parts: Training and Testing;
2. Train a model with the training data set; and
3. Test the trained model with the testing data set.

The following sections will illustrate these steps.

### <font color = black> Steps for Predictive Modeling: Train/Test Split Data </font>

The data are split into two parts using *sklearn*.  Each part has a *X* variable array and a *y* vector (The upper and lower cases are conventional).  The *X* array is a Pandas DataFrame of the *X* variables.  The *y* vector is a Pandas Series.

In [ ]:
##
## Import train_test_split package
##
from sklearn.model_selection import train_test_split

In [ ]:
##
## Create the X and y data for splitting
##
y = df[ 'Usales' ]
x = [ 'Pprice', 'Ddisc', 'Odisc', 'Cdisc', 'Pdisc', 'Region', 'buyerRating' ]
X = df[ x ]
##
## Split the data.  The default is 3/4 train.
##
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.25,
                                                    random_state = 42 ) 

**_Code Explanation_**

The dependent and independent variables need to be separated from the main DataFrame before the train/test split can be done.  The index from the main DataFrame is preserved.  The first three lines of code do this.  The *train_test_split* function randomly divides the data, keeping the indexes aligned.  The *random_state = 42* argument sets the random seed.  Four data sets are returned which are (in order): *X_train, X_test, y_train*, and *y_test*.

In [ ]:
##
## Display some data
##
print("Sample sizes: \nX: {}, y: {}\n".format( X_train.shape[0], y_test.shape[0] ) )
print( 'Training Data: \n{}'.format( X_train.head() ) )
print( "\n" )
print( 'Testing Data: \n{}'.format( y_test.head() ))

**_Interpretation_**

Note the indexes for the training and testing data sets.  These are the same as the main DataFrame, *df*.

In [ ]:
## 
## Merge the X and y training data for 
## model training.  Do an inner join on the indexes.
##
## Rename the y variable: Usales
##
yy = pd.DataFrame( { 'Usales':y_train } )
train = yy.merge( X_train, left_index = True, right_index = True )
print( 'Training Data Set:\n\n{}'.format( train.head() ) )

**_Code Explanation_**

The *X* and *Y* training data sets are merged on the indexes.  Recall that the index were preserved when the *y* and *X* data sets were created.  This is why.

In [ ]:
## 
## Merge the X and y testing data sets for predicting.
## Use an inner join on the indexes.
##
## Rename the y variable Usales.
##
yy = pd.DataFrame( { 'Usales':y_test } )
test = yy.merge( X_test, left_index = True, right_index = True )
print( 'Testing Data Set:\n\n{}'.format( test.head() ) )

In [ ]:
##
## Add log Usales and log Pprice to the training data
## The log is based on the Numpy function log1p
## Note: log1p( x ) = log( 1 + x )
##
train[ 'log_Usales' ] = np.log1p( train.Usales )
train[ 'log_Pprice' ] = np.log1p( train.Pprice )
print( 'Training Data Set:\n\n{}'.format( train.head() ) )
print( "\n" )
print( 'Training Data Set Shape:\n {}'.format( train.shape ) )

**_Code Explanation_**

Logged terms are added because the Data Visualization showed that logs induce normality.

In [ ]:
##
## Repeat for the testing data
##
test[ 'log_Usales' ] = np.log1p( test.Usales )
test[ 'log_Pprice' ] = np.log1p( test.Pprice )
print( 'Testing Data Set:\n\n{}'.format( test.head() ) )
print( "\n" )
print( 'Testng Data Set Shape:\n {}'.format( test.shape ) )

### <font color = black> Steps for Predictive Modeling: Training a Model </font>

I will cover three predictive models:

1. *OLS*
2. Logit
3. Decision trees

Which one is used depends on the dependent variable which can be continuous or discrete.  There are three cases corresponding to the two predictive models:

- Case I: Continuous Dependent Variable -- *OLS* Regression
- Case II: Binary Dependent Variable -- Logistic Regression
- Case III: Constants -- Decision Tree

#### <font color = black> Case I: Continuous Dependent Variable -- *OLS* Regression </font>

Model unit sales as a function of the pocket price to get a price elasticity.  Recall that you are using log terms and that the estimated coefficient for log price is the elasticity.

**Recommendation**:  Use formulas to specify the model.  You need the *statsmodels.formula* api for this.

In [ ]:
## 
## OLS
##
## For modeling, notice the new import command for
## the formula API and the summary option
##
import statsmodels.api as sm
import statsmodels.formula.api as smf 
##
## There are four steps for estimatng a model:
##   1. define a formula (i.e., the specific model to estimate)
##   2. instantiate the model (i.e., specify it)
##   3. fit the model
##   4. summarize the fitted model
##
## ===> Step 1: Define a formula
##
## The formula uses a “~” to separate the left-hand side from the right-hand side
## of a model and a “+” to add columns to the right-hand side.  A “-” sign (not 
## used here) can be used to remove columns from the right-hand side (e.g.,
## remove or omit the constant term which is always included by default). 
##
formula = 'log_Usales ~ log_Pprice + Ddisc + Odisc + Cdisc + Pdisc + C( Region )'
##
## Since Region is categorical, you must create dummies for the regions.  You
## do this using 'C( Region )' to indicate that Region is categorical.
##
## ===> Step 2: Instantiate the OLS model
##
mod = smf.ols( formula, data = train )
##
## ===> Step 3: Fit the instantiated model
##      Recommendation: number your models
##
reg01 = mod.fit()
##
## ===> Step 4: Summarize the fitted model
##
print( reg01.summary() )

**_Code Explanation_**

The modeling follows four steps as shown above.  Regardless of the software you might use, these same four steps are followed.  Some software combines them, others require explicit statement.  This is what statsmodels requires.

**_Interpretation_**

The price elasticity is the coefficient for the logged price variable (i.e., log_Pprice): -1.7.  If price falls by 1\%, unit sales rise by 1.7\%.  This indicates that blinds are highly elastic.  This should be expected since furniture is a competitive business and blinds are very competitive.  Revenue will also change.  If price fall, revenue will increase.  The amount revenue will increase (in percentage terms) is given by $1 + elasticity$.  So for a 1\% fall in price, revenue will rise 0.7\% (= $1 + [-1.7]$). 

The discounts and regions seem to have no effect, but this can be tested as we'll do below.  Also note that the $R^2$ is 0.20 which is very low.  

The Jarque-Bera Test is a test for normality of the disturbance term.  It is a test of the "goodness-of-fit test of whether sample data have the skewness and kurtosis matching a normal distribution. $\ldots$ The null hypothesis is a joint hypothesis of the skewness being zero and the excess kurtosis being zero.  $\ldots$ If it is far from zero, it signals the data do not have a normal distribution."  So the Null Hypothesis is $H_O: Normality$.  (Source: <a href="https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test" target="_parent">see here</a>)  In this case, the Null is rejected.  The Omnibus Test is an alternative test of normality with the same Null.  It also indicates that the Null must be rejected.

### <font color = blue> Exercises </font>

#### <font color = black> Exercise \#3.1 </font>

Estimate a new *OLS* model by adding the buyer rating to the above model. Interpret your results.  Is the buyer rating important for sales?

**Hint**: Buyer rating is categorical so you have to create dummies for the rating.

In [ ]:
##
## Enter code here
##


#### <font color = black> Analyzing the Results </font>

Quantities of interest can be extracted directly from the fitted model. Type *dir(results)* for a full list.

Since the product manager wanted to know about a region effect, you should do an F-test of all the coefficients for the regions to determine if they are all zero, meaning that the dummies as a group do nothing.  This is a <u>joint</u> test of significance.  The test statistic is:

$F_C = \dfrac{\left(SSR_U - SSR_R\right)/(df_U - df_R)}{SSE_U/(n - p - 1)} = \dfrac{\left(SSE_R - SSE_U\right)/(df_U - df_R)}{SSE_U/(n - p - 1)}$

where "U" indicates the *unrestricted* or *full* model with the Region dummies and "R" indicates the *restricted* model without the Region dummies.

In [ ]:
##
## Specify the joint (Null) hypothesis that the regions are the same;
## i.e., there is no region effect.
##
hypothesis = ' ( C(Region)[T.Northeast] = 0, C(Region)[T.South] = 0, C(Region)[T.West] = 0 ) '
##
## Run and print an F-test 
##
f_test = reg01.f_test( hypothesis )
f_test.summary()

**_Code Explanation_**

Notice that there are only three regions specified even though there are four: one is omitted as the base.  Also notice that the three hypotheses are specified as *C(Region)[T.XX] = 0* where *XX* is the region name.

**_Output Interpretation_**

The returned values for the F-test are, in order:

1. The F-Statistic value
2. The p-value for the F-Statistic
3. The F-Statistic's denominator degrees-of-freedom
4. The F-Statistic's numerator degrees-of-freedom

**_Interpetation_**

The Null Hypothesis is that there is no region effect.  The p-value is 0.32 so the Null Hypothesis is not rejected: there is no Region effect.

In [ ]:
##
## Repeat the F-test for the discounts
##
hypothesis = ' ( Ddisc = 0, Odisc = 0, Cdisc = 0, Pdisc = 0 ) '
##
## Run and print an F-test 
##
f_test = reg01.f_test( hypothesis )
f_test.summary()

**_Interpretation_**

The hypothesis statement does not have the discount names as *C(Ddis)* etc. because they are quantitative variables, not categorical variables like *Region*. 

The Null Hypothesis is that there is no difference among the discounts; they all have zero effect on unit sales.  Notice that the p-value is 0.34.  So the Null Hypothesis that the discounts all have the same effect is not rejected.

## <font color = blue> Exercises </font>

### <font color = black> Exercise \#3.2 </font>

Test the Null Hypothesis that all the buyer rating estimated parameters are zero.  That is, there is no difference among the ratings.

In [ ]:
##
## Enter code here
##


Check for multicollinearity -- a linear relationship among the variables.  Use the *variance inflation factor* (*VIF*).  A rule-of-thumb is that any $VIF > 10$ indicates a problem.

In [ ]:
##
## Subset the design matrix to eliminate the first column of 1s
## the iloc method says to find the location of columns based on 
## their integer locations (i.e., 0, 1, 2, etc.)
## the term in brackets says to find all rows (the : ) and all 
## columns from the first to the end (1: )
##
## Create the correlation matrix
##
x = reg01.model.data.orig_exog.iloc[ :, 1: ] 
corr_matrix = x.corr()
corr_matrix

In [ ]:
##
## Graph the correlation matrix
##
sns.heatmap( corr_matrix ).set_title( 'Heatmap of the Correlation Matrix' )

In [ ]:
## 
## A fancy version of the heatmap
## Based on: https://stackoverflow.com/questions/39409866/correlation-heatmap
##
cmap = sns.diverging_palette( 5, 250, as_cmap = True )
##
corr_matrix.style.background_gradient( cmap, axis=1 ).set_precision( 1 )


In [ ]:
##
## Calculate VIFs
## The VIFs are the diagonal elements of the inverted correlation
## matrix of the independent variables.
##
## Subset the design matrix to eliminate the first column of 1s.
## The iloc method says to find the location of columns based on their 
## integer locations (i.e., 0, 1, 2, etc.) the term in brackets says 
## to find all rows (the : ) and all columns from the first to the end (1: ).
##
## Create the correlation matrix
##
x = reg01.model.data.orig_exog.iloc[ :, 1: ]
corr_matrix = x.corr()
##
## Invert the correlation matrix and extract the main diaginal
##
vif = np.diag( np.linalg.inv( corr_matrix ) ) 
##
## Zip the variable names and the VIFs
##
indepvars = [ i for i in x.columns ]
xzip = zip( indepvars, vif ) 
##
## Display the zip matrix.  First import a needed function:
##
from statsmodels.compat import lzip
lzip( xzip )

**_Interpretation_**

The *VIF*s are all below 10 so there is no problem.  $VIF > 10$ is a rule-of-thumb for indicating multicollinearity.

#### Model Portfolio

This is a nice way to summarize the models.

In [ ]:
##
## Import some packags
##
from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.api import anova_lm
##
## Create a variable to hold the model names; this is a list.
## Note: the range() function specifies 1 - 2 but the "2" is
## not included.
##
model_names = [ 'Model ' + str( i ) for i in range( 1, 2 ) ]
##
## Create a variable to hold the statistics to print; this is a dictionary.
##
info_dict = { '\nn': lambda x: "{0:d}".format( int( x.nobs ) ),
              'R2 Adjusted': lambda x: "{:0.3f}".format( x.rsquared_adj ),
              'AIC': lambda x: "{:0.2f}".format( x.aic ),
              'F': lambda x: "{:0.2f}".format( x.fvalue ),
}
##
## Create the portfolio summary table.
##
summary_table = summary_col( [ reg01 ],
            float_format = '%0.2f',
            model_names = model_names,
            stars = True, 
            info_dict = info_dict 
)
summary_table.add_title( 'Summary Table for Living Room Blinds Sales' )
print( summary_table )


#### <font color = black> Predicting with the Model </font>

Predict unit sales.  Recognize that sales are in (natural) log terms.  You will convert back to unit sales in "normal" terms later.

In [ ]:
##
## Calculate predicted log of unit sales, the dependent variable.
##
## Note: the inverse of the log is needed; use np.expm1( x )
## since log1p was used: np.expm1 = exp(x) - 1.
##
log_pred = reg01.predict( test )
y_pred = np.expm1( log_pred )
##
##
## Combine into one temporary DataFrame for convenience
##
tmp = pd.DataFrame( { 'y_test':y_test, 'y_logPred':log_pred, 'y_pred':y_pred } )
tmp.info()

Use the sklearn metrics function *r2_score* to check the fit of actual vs. predicted values.  From the sklearn User Guide:

"*The r2_score function computes R², the coefficient of determination. It provides a measure of how well future samples are likely to be predicted by the model. Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R^2 score of 0.0.*"

In [ ]:
##
## Import the r2_score function from the sklearn metrics package
##
from sklearn.metrics import r2_score
##
## Display the r2 score.  But first drop any NaN data.
##
tmp.dropna( inplace = True )
print( 'r2 Score:\n {}'. format( round( r2_score( tmp.y_test, tmp.y_pred), 3 ) ) )

This is not very good.

You can also graph the actual vs predicted values.  Sometimes, however, the number of data points is too large to plot so a random sample may be needed.  This is our case.

In [ ]:
##
## Draw a random sample of 500 observations without replacement
## from the tmp DataFrame.
##
smpl = tmp.sample( n = 500, replace = False, random_state = 1 )
##
## Plot the data
##
ax = sns.regplot( x = 'y_test', y = 'y_pred', scatter = True, data = smpl )
ax.set( title = 'Actual vs Predicted Units Sales\nRandom Sample of 500', 
       ylabel = 'Predicted Sales', xlabel = 'Actual Sales' )

Predict unit sales for different settings of the variables.  This is *scenario* or *what-if* analysis.

In [ ]:
##
## Specify scenario values to use for prediction
##
## Create a dict
##
data = {
         'Pprice': [ 2.50 ],
         'Ddisc': [ 0.03 ],
         'Odisc': [ 0.05 ],
         'Cdisc': [ 0.03 ],
         'Pdisc': [ 0.03 ],
         'Region': [ 'West' ]
        }
##
## Create a DataFrame using the dict
##
scenario = pd.DataFrame.from_dict( data )
##
## Insert a log price column after the Pprice variable
##
scenario.insert( loc = 1, column = 'log_Pprice',
                value = np.log1p( scenario.Pprice ) )
##
## Display the settings and the predicted unit sales
##
print( 'Scenario settings:\n{}'.format( scenario ) )
##
## Create a pediction
##
log_pred = reg01.predict( scenario )
y_pred = np.expm1( log_pred )
print( '\nPredicted Unit Sales: \n{}'.format( round( y_pred, 0 ) ) )

#### <font color = black> Case II: Binary Dependent Variable -- Logistic Regression </font>

#### <font color = black> Create your Data </font>

Customer satisfaction is part of the DataFrame.  Satisfaction is measured on a five-point scale: *1 = Not at All Satisfied*, *5 = Very Satisfied*.  

First, look at the frquency count of satisfaction.  But, there is a problem: you cannot use the same data as before since satisfaction is by customer and the data used so far are by transaction.  The satisfaction rating is in the customer DataFrame.  You need to first find the mean price and mean discounts by customer from the transactions DataFrame and then merge this new DataFrame with the customer DataFrame.  So, there are several steps:

1. Extract the pocket price and discounts -- include the *CID*
2. Group by the CID and calculate the means by *CID*
3. Merge with the customer DataFrame
4. Recode the scale values in the merged file so that 1 is the top-two values (called *top-two box* or *T2B*) and 0 is all other values.  The *T2B* is *Very Satisfied*.
5. Train a model with *T2B* satisfaction as a function of the pocket price, discounts, and Region.

In [ ]:
## 
## ===> Step 1: Extract the pocket price and discounts -- include the CID
##
tmp = df[ [ 'CID', 'Pprice', 'Ddisc', 'Odisc', 'Cdisc', 'Pdisc' ] ]
tmp.set_index( 'CID', inplace = True )
tmp.shape

In [ ]:
##
## ===> Step 2: Group by the CID and calculate the means by CID
##
x = tmp.groupby( 'CID' ).mean()
x.shape

In [ ]:
##
## ===> Step 3: Merge with the customer DataFrame
##
df_sat = x.merge( df_cust, left_index = True, right_index = True )
df_sat.head()
df_sat.shape

In [ ]:
##
## Do a quick check of the value distribution.
##
## Use the DataFrame's value_counts() method. Sort by the
## scale values 1 - 5.
##
df_sat.buyerSatisfaction.value_counts( sort = False )

In [ ]:
##
## ===> Step 4: Recode the scale values so that 1 is the top-two values 
## (called "top-two box" or "T2B") and 0 is all other values.  
## The "T2B" is "Very Satisfied".
##
## Recode using Numpy's select function
##
## ===> Step 4.A: Define labels for the recoded values
##
lbl = [ 1, 0 ]
##
## ===> Step 4.B: Specify the conditions for the recoding
##
conditions = [
    ( df_sat.buyerSatisfaction >= 4 ),
    ( df_sat.buyerSatisfaction < 4 )
]
##
## ===> Step 4.C: Do the recoding 
##
df_sat[ 'sat_t2b' ] = np.select( conditions, lbl )
##
df_sat[ 'sat_t2b' ].value_counts( normalize = True )

Model *T2B* satisfaction as a function of the pocket price and discounts.  First, create training and testing DataFrames as before but with *sat_t2b* as the *y* variable.

In [ ]:
##
## ===> Step 5: Train a model.
##
## Create the X and y data for splitting
##
y = df_sat[ 'sat_t2b' ]
x = [ 'Pprice', 'Ddisc', 'Odisc', 'Cdisc', 'Pdisc', 'Region' ]
X = df_sat[ x ]
##
## Split the data.  The default is 3/4 train.
##
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, 
                                                    random_state = 42 )

In [ ]:
##
## Display some data
##
print( 'Training Data: \n{}'.format( X_train.head() ) ) 
print( "\n" )
print( 'Testing Data: \n{}'.format( y_test.head() ) )

In [ ]:
## 
## Merge the two training sets for convenience
##
yy = pd.DataFrame( { 'sat_t2b':y_train } )
train = yy.merge( X_train, left_index = True, right_index = True )
train.head()

In [ ]:
## 
## Merge the two testing sets for convenience
##
yy = pd.DataFrame( { 'sat_t2b':y_test } )
test = yy.merge( X_test, left_index = True, right_index = True )
test.head()

####  <font color = black> Train a Model </font>

In [ ]:
##
## Train a logit model
##
## ===> Step 1: Define a formula
##
formula = 'sat_t2b ~ Pprice + Ddisc + Odisc + Cdisc + Pdisc + C( Region )'
##
## ===> Step 2: Instantiate the logit model
##
mod = smf.logit( formula, data = train )
##
## ===> Step 3: Fit the instantiated model
##
logit01 = mod.fit()
##
## ===> Step 4: Summarize the fitted model
##
print( logit01.summary() )

#### <font color = black> Analyze the Results </font>

In [ ]:
##
## Import needed functions
##
from sklearn.metrics import confusion_matrix, classification_report
##
## Make predictions
##
predictions = logit01.predict( test )
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]
print( classification_report( y_test, predictions_nominal, digits = 3 ) )

For binary classification, the count of **true negatives** ($tn$), **false negatives** ($fn$), **true positives** ($tp$), and **false positives** ($fp$) can be found from a *confusion matrix*.

In [ ]:
##
## Create a confusion matrix
##
x = confusion_matrix(y_test, predictions_nominal).ravel()
##
## zip the variable names and the confusion
##
lbl = [ 'tn', 'fp', 'fn', 'tp' ]
##
## display the zip matrix
##
from statsmodels.compat import lzip
lzip( zip( lbl, x ) )


**_Interpretation_**

There were 4 true negatives, 23 false positives, 2 false negatives, and 72 true positives.

Plot the confusion matrix.

In [ ]:
##
## Create labels
##
lbl = ['Not Satisfied', 'Satisfied']
##
## Create the confusion matrix
##
cm = confusion_matrix( y_test, predictions_nominal )
tmp = pd.DataFrame(data=cm, index = lbl, columns = lbl )
print( 'Confusion Matrix: \n{}'.format( tmp ) )
##
## Plot the confusion matrix
##
sns.set( font_scale = 1.4 )   #for label size
##
ax = sns.heatmap( cm/cm.sum(), annot = True, annot_kws = { "size": 16 } )  # font size
ax.set( title = 'Confusion Matrix of the Classifier', xlabel = 'Predicted',
       ylabel = 'True' )
ax.set_xticklabels(lbl)
ax.set_yticklabels(lbl)

**_Interpretation_**

75\% of the cases were predicted correctly.

#### Model Portfolio

In [ ]:
model_names = [ 'Model ' + str( i ) for i in range( 1, 2 ) ]
##
## Create a variable to hold the statistics to print; this is a dictionary.
##
info_dict = { '\nn': lambda x: "{0:d}".format( int( x.nobs ) ),
}
##
## Create the portfolio summary table.
##
summary_table = summary_col( [ logit01 ],
            float_format = '%0.2f',
            model_names = model_names,
            stars = True, 
            info_dict = info_dict 
)
summary_table.add_title( 'Summary Table for Living Room Blinds Sales' )
print( summary_table )

#### <font color = black> Predicting with the Model </font>

The prediction process is the same as discussed for *Case I* above.

#### <font color = black> Case III: Constants - Decision Trees </font>

Decision Trees can handle continuous or discrete dependent variables.  They are an alternative to *OLS* and logistic regression: you don't have to specify a "model" *per se*.  They also have the advantage that a visual display, a *tree*, is produced which is easier for management and clients to understand than complex regression output and statistics.  You will only look at a discrete case; a continuous case is the same.

In [ ]:
##
## Import decision tree classifier
##
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing
from sklearn.tree import export_graphviz
##
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
##
## Convert "Region" to integers: the decision tree must have all numerics
## Note 1: use the LabelEncoder function for this
## Note 2: "Region" will be encoded in alphanumeric order:
##
##          0: Midwest
##          1: Northeast
##          2: South
##          3: West
##
print( '\nTraining Data before recoding Region:\n \n{}'.format( X_train.head() ) )
le = preprocessing.LabelEncoder()
X_train[ 'Region' ] = labelencoder.fit_transform( X_train[ 'Region'] )
print( '\nTraining Data after recoding Region:\n \n{}'.format( X_train.head() ) )
##
X_test = X_test.apply( le.fit_transform )
print( 'Testing Data: \n{}'.format( X_test.head() ) )
##
## Instantiate the tree
##
dtree = tree.DecisionTreeClassifier( random_state = 0, max_depth = 3, 
                                    min_samples_leaf = 5 )
##
## Fit the tree
##
dtree.fit( X_train, y_train )

Some additional packages are needed to plot a decision tree:

- graphviz
- pydotplus


In [ ]:
##
## Both packages may have to be installed before they can be used.  
## Use the operating system to do this.
##
import os
!{sys.executable} -m pip install graphviz
!{sys.executable} -m pip install pydotplus
##
## Tell Python where the graphviz package is load; then load it.
##
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
##
## Load the following packages
##
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import graphviz
import pydotplus

#### <font color = black> Check Model Accuracy </font>

In [ ]:
##
## The score attribute
##
print( "Accuracy on training data: {:.3f}".format( dtree.score( X_train, y_train )))
print( "Accuracy on testing data: {:.3f}".format( dtree.score( X_test, y_test )))

These are good scores.

#### <font color = black> Display the Decision Tree </font>

In [ ]:
##
## Import needed packages
##
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
##
## Displaying a tree is a slight challenge!
## There are four steps:
##
## ===> Step 1: Create a placeholder for all the plotting points.
##
dot_data = StringIO()
##
## ===> Step 2: Extract the feature names for labels models
##
feature_names = [ i for i in X_train.columns ]
##
## ===> Step 3: Export the plotting data to the placeholder
##
export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                class_names = [ 'Not Satisfied', 'Satisfied' ],
                feature_names = feature_names ,
                proportion  = True
               )
##
## ===> Step 4: Create the display
##
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

**_Note_**

For the right node on the third level, "$Region \le 0.05$" is interpreted as *Region* having a value less than or equal to 0.05.  Since only $Region = 0$ meets this criteria and $Region = 0$ is the Midwest, than if *Region* is the Midwest, go to the left; otherwise, go to the right for regions Northest, South, and West. 

## <font color = black> Exercise \#3.3 </font>

Interpret the decision tree.

## <font color = black> What's Next? </font>

In Lesson 4, I will briefly discuss how to share your notebooks.  Recall that there are three reasons for using Jupyter notebooks in your analysis work:

1. managing workflows;
2. documenting analyses for reproducibility; and
3. sharing results with colleagues.

I'll discuss these in the next lesson.
<br><br><br>
<font color = red, size = "+3"><b> Five Minute Break </font>